# Weather Data Analysis for Irish Locations

## 1. Introduction

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 2. Data Sources and File Paths

In [76]:
# File paths and corresponding locations
file_paths = [
    'data/cork_airport.csv',
    'data/dublin_airport.csv',
    'data/knock_airport.csv',
    'data/malin_head.csv',
    'data/mullingar.csv'
]
locations = ['Cork', 'Dublin', 'Knock', 'Malin Head', 'Mullingar']

## 3. Data Loading and Preparation

In [77]:
# Load each file and add the 'Location' column
cork_data = pd.read_csv(file_paths[0], skiprows=19) 
cork_data['Location'] = 'Cork'

dublin_data = pd.read_csv(file_paths[1], skiprows=19)
dublin_data['Location'] = 'Dublin'

knock_data = pd.read_csv(file_paths[2], skiprows=19)
knock_data['Location'] = 'Knock'

malin_data = pd.read_csv(file_paths[3], skiprows=19)
malin_data['Location'] = 'Malin Head'

mullingar_data = pd.read_csv(file_paths[4], skiprows=19)
mullingar_data['Location'] = 'Mullingar'

[Reference: Add column with constant value to pandas dataframe](https://www.geeksforgeeks.org/add-column-with-constant-value-to-pandas-dataframe/)<br/>


## 4. Consolidating Data

### Column Descriptions

- **year**: Year  
- **month**: Month  
- **rain**: Precipitation Amount (mm)  
- **meant**: Mean Air Temperature (°C)  
- **maxtp**: Maximum Air Temperature (°C)  
- **mintp**: Minimum Air Temperature (°C)  
- **mnmax**: Mean Maximum Temperature (°C)  
- **mnmin**: Mean Minimum Temperature (°C)  
- **gmin**: Grass Minimum Temperature (°C)  
- **wdsp**: Mean Wind Speed (knot)  
- **mxgt**: Highest Gust (knot)  
- **sun**: Sunshine Duration (hours)  


In [78]:

# Concatenate all DataFrames
all_data = pd.concat([cork_data, dublin_data, knock_data, malin_data, mullingar_data], ignore_index=True)

# Save the consolidated data to an SQLite database
conn = sqlite3.connect('data/weather_data.db')
all_data.to_sql('consolidated_weather', conn, if_exists='replace', index=False)

# Confirm the data was stored correctly
print("Preview of consolidated data in SQLite:")
print(pd.read_sql_query("SELECT * FROM consolidated_weather LIMIT 5", conn))


Preview of consolidated data in SQLite:
   year  month  meant maxtp mintp  mnmax  mnmin  rain  gmin  wdsp maxgt  \
0  1962      1    5.5  11.6  -3.8    8.2    2.7        -6.3  12.8    70   
1  1962      2    5.3  12.3  -2.7    7.9    2.7        -4.4  13.6    57   
2  1962      3    4.2  12.2  -5.4    7.5    0.9        -7.2  10.3    63   
3  1962      4    7.7  18.8   0.6   11.2    4.1  68.6  -1.3  11.7    62   
4  1962      5    9.7  16.1   2.3   13.3    6.2  80.7  -0.7  12.0    45   

     sun Location  
0            Cork  
1            Cork  
2            Cork  
3  201.1     Cork  
4  216.5     Cork  


[Reference: pandas.concat() function in Python](https://www.geeksforgeeks.org/pandas-concat-function-in-python/)<br/>
[Reference: How to write Pandas dataframe to sqlite with Index](https://stackoverflow.com/questions/14431646/how-to-write-pandas-dataframe-to-sqlite-with-index)<br/>
[Reference: Accessing SQLite Databases Using Python and Pandas](https://datacarpentry.github.io/python-ecology-lesson/instructor/09-working-with-sql.html)<br/>
[Reference: Pandas Read SQL Query or Table with Examples](https://sparkbyexamples.com/pandas/pandas-read-sql-query-or-table/)<br/>
[Reference: MySQL LIMIT Clause](https://www.w3schools.com/mysql/mysql_limit.asp)<br/>

In [79]:

# Query to filter data for year >= 2000 and month >= 1
query = """
SELECT * 
FROM consolidated_weather
WHERE year >= 2000 AND month >= 1
"""
analysis_weather_records = pd.read_sql_query(query, conn)

# Confirm the filtered data
print("Filtered data preview:")
print(analysis_weather_records.head())



Filtered data preview:
   year  month  meant maxtp mintp  mnmax  mnmin   rain  gmin  wdsp maxgt  \
0  2000      1    5.6  12.5  -2.0    8.0    3.1   50.2  -7.0  10.1    48   
1  2000      2    6.8  12.7   0.0    9.9    3.7  112.0  -3.6  12.0    50   
2  2000      3    8.0  15.7   0.4   11.3    4.7   20.2  -5.2   7.6    38   
3  2000      4    7.3  14.4  -1.8   10.6    3.9   50.2  -6.7  10.1    45   
4  2000      5   11.7  22.1   4.0   15.6    7.8   70.8   0.0   9.1    38   

     sun Location  
0   77.1     Cork  
1   85.6     Cork  
2  122.3     Cork  
3  147.0     Cork  
4  237.8     Cork  


[Reference: The SQL WHERE Clause](https://www.w3schools.com/sql/sql_where.asp)<br/>
[Reference: MySQL AND, OR and NOT Operators](https://www.w3schools.com/mysql/mysql_and_or.asp)<br/>

## 6. Handling Missing Values

In [80]:

# Replace blank fields and '-' with NaN in the filtered data
analysis_weather_records.replace([r'^\s*$'], np.nan, regex=True, inplace=True)

# Count and display missing values grouped by location
missing_values = analysis_weather_records.isnull().groupby(analysis_weather_records['Location']).sum()
print("Missing values by column and location:")
print(missing_values)

Missing values by column and location:
            year  month  meant  maxtp  mintp  mnmax  mnmin  rain  gmin  wdsp  \
Location                                                                       
Cork           0      0      0      0      0      0      0     0     0     0   
Dublin         0      0      0      0      0      0      0     0     1     0   
Knock          0      0      0      0      0      0      0     0     0     0   
Malin Head     0      0      0      0      0      0      0     0     0     1   
Mullingar      0      0      0      0      0      0      0     0     1     0   

            maxgt  sun  Location  
Location                          
Cork            0    0         0  
Dublin          0    0         0  
Knock           0   55         0  
Malin Head      0  166         0  
Mullingar       0  214         0  



[Reference: Replace values in Pandas dataframe using regex](https://www.geeksforgeeks.org/replace-values-in-pandas-dataframe-using-regex/)<br/>
[Reference: Pandas: How to Replace Zero with NaN](https://www.statology.org/pandas-replace-0-with-nan/)<br/>
[Reference: A Guide to R Regular Expressions](https://www.datacamp.com/tutorial/regex-r-regular-expressions-guide) <br/>
[Reference: Using isnull() and groupby() on a pandas dataframe](https://stackoverflow.com/questions/46106954/using-isnull-and-groupby-on-a-pandas-dataframe)<br/>
[Reference: pd.NA vs np.nan for pandas](https://stackoverflow.com/questions/60115806/pd-na-vs-np-nan-for-pandas)<br/>



### Overview of Missing Data

- **Data Collection Gaps for `sun`**:
  - No data recorded in **Mullingar** since January 2007.
  - No data recorded in **Malin Head** since January 2011.
  - No data recorded in **Knock** since January 2020.

- **Specific Missing Values**:
  - **Dublin**: 1 missing `gmin` value.
  - **Mullingar**: 1 missing `gmin` value.
  - **Malin Head**: 1 missing `wdsp` value.

**Exclusions from Analysis**:
  - The `sun` and `gmin` columns will be excluded from the analysis as they are not relevant to the goals of this project.
  - One missing `wdsp` value will be dropped, as it is negligible in the overall dataset.


In [81]:
# Display the data types of all columns to verify they are correctly formatted for analysis
print(analysis_weather_records.dtypes)

year          int64
month         int64
meant       float64
maxtp        object
mintp        object
mnmax       float64
mnmin       float64
rain         object
gmin         object
wdsp         object
maxgt        object
sun          object
Location     object
dtype: object


In [82]:
# Convert selected columns to numeric data types, coercing invalid entries to NaN for cleaning
columns_to_convert = ['maxtp', 'mintp', 'rain', 'gmin', 'wdsp', 'maxgt', 'sun']
for col in columns_to_convert:
    analysis_weather_records[col] = pd.to_numeric(analysis_weather_records[col], errors='coerce')

[Reference: pandas: to_numeric for multiple columns](https://stackoverflow.com/questions/36814100/pandas-to-numeric-for-multiple-columns)<br/>

In [83]:
# Drop rows with missing 'wdsp' values
analysis_weather_records.dropna(subset=['wdsp'], inplace=True)

In [84]:
# Check the data types of all columns after conversion to ensure they are now correctly formatted
print(analysis_weather_records.dtypes)

year          int64
month         int64
meant       float64
maxtp       float64
mintp       float64
mnmax       float64
mnmin       float64
rain        float64
gmin        float64
wdsp        float64
maxgt         int64
sun         float64
Location     object
dtype: object


[Reference: Pandas: How to Use dropna() with Specific Columns](https://www.statology.org/pandas-dropna-specific-column/)<br/>

## 7. Wind Analysis

Line plot with markers showing the monthly mean wind speed for each location.

In [98]:

# Extract relevant columns and create a copy to avoid modifying the original DataFrame
monthly_windspeed_data = analysis_weather_records[['Location', 'year', 'month', 'wdsp']].copy()

# Add a 'date' column and convert to datetime
monthly_windspeed_data['date'] = pd.to_datetime(
    monthly_windspeed_data[['year', 'month']].assign(day=1)
)



# Create subplots for each location
locations = monthly_windspeed_data['Location'].unique()
fig = make_subplots(
    rows=len(locations), cols=1, shared_xaxes=True,
    subplot_titles=[f"Average Windspeed - {loc}" for loc in locations]
)

# Plot wind speed trends for each location
for i, location in enumerate(locations):
    fig.add_trace(
        go.Scatter(
            x=monthly_windspeed_data[monthly_windspeed_data['Location'] == location]['date'],
            y=monthly_windspeed_data[monthly_windspeed_data['Location'] == location]['wdsp'],
            mode='lines+markers',
            name=location
        ),
        row=i + 1, col=1
    )

# Configure figure layout and shared axis settings
fig.update_layout(
    height=300 * len(locations),
    
    template="plotly_white",
    showlegend=False
)
fig.update_xaxes(title_text="Date", matches='x')
fig.update_yaxes(title_text="Average Windspeed (knots)", matches='y1')

# Render the final plot
fig.show()






[Reference: Plotly- Subplots in Python](https://plotly.com/python/subplots/?utm_source=chatgpt.com)<br/>
[Reference:Plotly- Scatter Plots in Python ](https://plotly.com/python/line-and-scatter/)<br/>
[Reference:Plotly- Python Figure Reference: layout ](https://plotly.com/python/reference/layout/)<br/>
[Reference:Plotly-Python Figure Reference: layout.xaxis ](https://plotly.com/python/reference/layout/xaxis/)<br/>
[Reference:Python Figure Reference: layout.yaxis ](https://plotly.com/python/reference/layout/yaxis/#:~:text=Python%20Figure%20Reference%3A%20layout.,yaxis&text=Type%3A%20dict%20containing%20one%20or%20more%20of%20the%20keys%20listed%20below.&text=If%20set%20to%20an%20opposite,the%20corresponding%20opposite%2Dletter%20axis.)<br/>
[Reference: Pandas DataFrame copy() Method](https://www.w3schools.com/python/pandas/ref_df_copy.asp)<br/>
[Reference: Pandas DataFrame assign() Method | Create new Columns in DataFrame](https://www.geeksforgeeks.org/pandas-dataframe-assign/)<br/>
[Reference: How to Specify Format in pandas.to_datetime](https://www.statology.org/pandas-to-datetime-format/) <br/>
[Reference: The Basics of the Python Enumerate() Function](https://blog.hubspot.com/website/python-enumerate)<br/>

In [54]:
# Close the SQLite connection
conn.close()
